In [7]:
import random
import numpy as np
from scipy.stats import bernoulli
import plotly.graph_objects as go
import networkx as nx
import matplotlib.pyplot as plt
import scipy as sp
import csv

In [8]:
def getCodToID():
  codToID = {}
  with open("/content/drive/MyDrive/Hackathon IA/Datasets/Brut Data/IdToCode.csv" , "r", newline="\n") as csvfile :
    csvreader = csv.reader(csvfile , delimiter = " ")
    for row in csvreader :
      codToID[int(row[0])] = int(row[1])
  
  return codToID

codToID = getCodToID()


def getAdjacencyMatrix():
  A = np.zeros(shape = (68 , 68) , dtype="int64")
  with open("/content/drive/MyDrive/Hackathon IA/Datasets/Brut Data/trip.csv" , "r") as csvfile :
    csvreader = csv.reader(csvfile)
    next(csvreader)
    for row in csvreader :
      if len(row) > 0:
        i = int(row[4])
        j = int(row[7])
        if i != 38 and i != 41 and j != 38 and j != 41:
          i = codToID[i]
          j = codToID[j]
          A[i][j] += 1
          A[j][i] += 1
    return A

A = getAdjacencyMatrix()

In [2]:
def getSubMatrix(M , label):
  a_eliminer = []
  for i in range(label.shape[0]):
    if label[i] == 1:
      a_eliminer.append(i)
  
  B = np.delete(M, a_eliminer , axis=0 )
  B = np.delete(B, a_eliminer , axis=1 )

  dico = {}
  count = 0
  for i in range(label.shape[0]):
    if label[i]:
      dico[count] = i
      count += 1
  return B , dico

In [3]:
san_francisco = [45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 63,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 62,
 65]

n_s_f = len(san_francisco)

In [4]:
label = [1 if i in san_francisco else 0 for i in range(68)]

In [14]:
B , dico = getSubMatrix(A, np.array(label))

In [5]:
from keras.layers import Input, Dense, Embedding
from keras.models import Model, load_model, Sequential

def Embedd8D():
  model = Sequential()
  model.add(Input(2))
  model.add(Embedding(n_s_f, 8, input_length=2))
  model.add(Dense(5, activation="relu"))
  model.add(Dense(1, activation="sigmoid"))

  return model

model = Embedd8D()
model.compile(loss = "binary_crossentropy" , metrics = "accuracy" , optimizer="adam")
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 2, 8)              272       
_________________________________________________________________
dense (Dense)                (None, 2, 5)              45        
_________________________________________________________________
dense_1 (Dense)              (None, 2, 1)              6         
Total params: 323
Trainable params: 323
Non-trainable params: 0
_________________________________________________________________


In [9]:
X = []
Y = []
with open("/content/drive/MyDrive/Hackathon IA/Datasets/Brut Data/trip.csv" , "r") as csvfile :
  csvreader = csv.reader(csvfile)
  next(csvreader)
  for row in csvreader:
    if int(row[4]) in codToID and int(row[7]) in codToID:
      i = codToID[int(row[4])]
      j = codToID[int(row[7])]
      if j in san_francisco and i in san_francisco:
        X.append([san_francisco.index(i) , san_francisco.index(i)])
        Y.append(1)
        X.append([random.randint(0,n_s_f-1) , random.randint(0,n_s_f-1)])
        Y.append(0)

X = np.array(X)
Y = np.array(Y)

In [10]:
model.fit(X , Y, epochs = 10, batch_size=8192, verbose=1)

Epoch 1/10
143/143 [==============================] - 2s 6ms/step - loss: 0.6833 - accuracy: 0.5762
Epoch 2/10
143/143 [==============================] - 1s 7ms/step - loss: 0.6652 - accuracy: 0.5993
Epoch 3/10
143/143 [==============================] - 1s 7ms/step - loss: 0.6637 - accuracy: 0.6001
Epoch 4/10
143/143 [==============================] - 1s 7ms/step - loss: 0.6641 - accuracy: 0.5994
Epoch 5/10
143/143 [==============================] - 1s 6ms/step - loss: 0.6637 - accuracy: 0.5999
Epoch 6/10
143/143 [==============================] - 1s 6ms/step - loss: 0.6636 - accuracy: 0.6001
Epoch 7/10
143/143 [==============================] - 1s 7ms/step - loss: 0.6639 - accuracy: 0.5996
Epoch 8/10
143/143 [==============================] - 1s 6ms/step - loss: 0.6635 - accuracy: 0.6006
Epoch 9/10
143/143 [==============================] - 1s 6ms/step - loss: 0.6640 - accuracy: 0.5993
Epoch 10/10
143/143 [==============================] - 1s 6ms/step - loss: 0.6637 - accuracy: 0.6000

In [11]:
model.save("/content/drive/MyDrive/Hackathon IA/Models/Data Mining/Embed256DModel")

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Hackathon IA/Models/Data Mining/Embed256DModel/assets
